In [1]:
import numpy as np
import pandas as pd

import patsy

# 1.Patsy：

### patsy.dmatrices函数接收一个公式字符串和一个数据集（可以是一个DataFrame或数组字典），并产生线性模型的设计矩阵

In [2]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],'x1': [1, -2, 3, -4, 0],'y': [-15, 0, 35, 15, -20]})
data

,x0,x1,y
0,1,1,-15
1,2,-2,0
2,3,3,35
3,4,-4,15
4,5,0,-20


In [3]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [4]:
y

DesignMatrix with shape (5, 1)
    y
  -15
    0
   35
   15
  -20
  Terms:
    'y' (column 0)

In [5]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  x1
          1   1   1
          1   2  -2
          1   3   3
          1   4  -4
          1   5   0
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

### 这些Patsy 的 DesignMatrix 实例是带有额外元数据的NumPy ndarray。

In [6]:
np.asarray(y)

array([[-15.],
       [  0.],
       [ 35.],
       [ 15.],
       [-20.]])

In [7]:
np.asarray(X)

array([[ 1.,  1.,  1.],
       [ 1.,  2., -2.],
       [ 1.,  3.,  3.],
       [ 1.,  4., -4.],
       [ 1.,  5.,  0.]])

### 输出结果中的 Intercept（截距 ）是普通最小二乘法（OLS）回归等线性模型中的惯例。你可以通过在模型中加入名词列 +0 来抑制截距。

In [8]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0  x1
   1   1
   2  -2
   3   3
   4  -4
   5   0
  Terms:
    'x0' (column 0)
    'x1' (column 1)

### Patsy对象可以直接传递给numpy.linalg.lstsq等算法，这些算法都会执行一个最小二乘回归。

### 模型元数据保留在 design_info 属性中，因此你可以将模型列名重新附加到拟合系数上，以获得一个Series。

In [9]:
coef, resid, _, _ = np.linalg.lstsq(X, y , rcond=-1)

In [10]:
coef

array([[0.39855072],
       [1.05072464],
       [1.37681159]])

In [11]:
resid

array([1975.18115942])

In [12]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.398551
x0           1.050725
x1           1.376812
dtype: float64

# 2.Patsy公式中的数据转换：

### 你可以将Python代码混入你的Patsy公式中；当执行公式时，Patsy库会尝试在封闭作用域中寻找你使用的函数。

In [13]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [14]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.69315
          1   2                 1.09861
          1   3                 1.38629
          1   4                 1.60944
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

### 一些常用的变量转换包括标准化（均值为0，方差为1）和居中（减去平均值）。Patsy有内置的函数来实现这一目的。

In [15]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [16]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421         1.4
          1         -0.70711        -1.6
          1          0.00000         3.4
          1          0.70711        -3.6
          1          1.41421         0.4
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

### patsy.build_design_matrices函数可以使用原始样本内数据集中保存的信息将变换应用于新的样本外数据上：

In [17]:
new_data = pd.DataFrame({'x0': [6, 7, 8, 9],'x1': [3, -0.5, 0, 2],'y': [1, 2, 3, 4]})
new_data

,x0,x1,y
0,6,3.0,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.0,4


In [18]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [19]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132         3.4
           1          2.82843        -0.1
           1          3.53553         0.4
           1          4.24264         2.4
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

### 因为加号（+）在Patsy公式的上下文中并不意味着加法，当你想对数据集中两列按列名相加，你必须用特殊的 I 函数来封装。

In [20]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [21]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1           2
          1           0
          1           6
          1           0
          1           5
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

# 3.分类数据与Patsy：

### 当你在Patsy公式中使用非数字名词列时，它们会被默认转换为虚拟变量。如果有一个拦截，其中一个级别将会被排除，以避免共线性。

In [22]:
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
                     'key2': [0, 1, 0, 1, 0, 1, 0, 0],
                     'v1': [1, 2, 3, 4, 5, 6, 7, 8],
                     'v2': [-1, 0, 2, -5, 4, -1, 2, -3]})
data

,key1,key2,v1,v2
0,a,0,1,-1
1,a,1,2,0
2,b,0,3,2
3,b,1,4,-5
4,a,0,5,4
5,b,1,6,-1
6,a,0,7,2
7,b,0,8,-3


In [23]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [24]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

### 如果你忽略了模型中的截距，那么每个类别值的列将被包括在模型的设计矩阵中。

In [25]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [26]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

### 数值列可以用C函数解释为分类列。

In [27]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [28]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

### 当你在一个模型中使用多个分类名词列时，事情可能会更复杂，因为你可以包括形式为key1:key2的交互项，例如，可以用于方差分析（ANOVA）模型。

In [29]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1
1,a,one,2,0
2,b,zero,3,2
3,b,one,4,-5
4,a,zero,5,4
5,b,one,6,-1
6,a,zero,7,2
7,b,zero,8,-3


In [30]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [31]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [32]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [33]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)